In [ ]:
! pip install pandas
! pip install pydicom

import glob, pylab, pandas as pd
import pydicom, numpy as np
from skimage.transform import resize



In [ ]:
# Show files in data
!ls data

# display label format
df = pd.read_csv('data/stage_1_train_labels.csv')


# 
No = 4
patientId = df['patientId'][No]
dcm_file = 'data/stage_1_train_images/%s.dcm' % patientId
dcm_data = pydicom.read_file(dcm_file)
#print(df.iloc[No])


#print(np.unique(df.Target))
#print(df['patientId'])

IMG_SIZE = 224
img_dimension = [IMG_SIZE,IMG_SIZE] # New size of xray images. 
new_df = pd.DataFrame({'Target':[],'Image':[]}) # Define new data frame for images and labels.
unq, idx = np.unique(df['patientId'], return_index = True) # Get only unique entrances from the provided data (some patients occur multiple times)

# Reshape images and match to corresponding label in new dataframe
for i in idx[0:1000]:
    new_df.Target[i] = df.Target[i] # Get label
    patientId = df['patientId'][i] # get patient id from the idx 
    dcm_file = 'data/stage_1_train_images/%s.dcm' % patientId # find the image-file corresponding to the patient id
    dcm_data = pydicom.read_file(dcm_file) # Load the image 
    new_df.Image[i] = resize(dcm_data.pixel_array, output_shape=img_dimension, mode='reflect', anti_aliasing=True) # resize image
    if not i % 100:
        print(i)
        
new_df.to_csv('data/data.csv')        

#Show an example image
im = new_df.Image[30]
pylab.imshow(im, cmap=pylab.cm.gist_gray)
pylab.axis('off')

In [ ]:
# -- Plotting --
    f, axarr = plt.subplots(2, 2, figsize=(20, 20))

    ######## Loss ########
    ax = axarr[0, 0]
    ax.set_title("Error")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Error')


    ax.plot(np.arange(epoch+1), train_loss, color="black")
    ax.plot(np.arange(epoch+1), valid_loss, color="gray", linestyle="--")
    ax.legend(['Training error', 'Validation error'])

    ######## Latent space ########
    ax = axarr[0, 1]

    ax.set_title('Latent space')
    ax.set_xlabel('Dimension 1')
    ax.set_ylabel('Dimension 2')
    
    # If you want to use a dimensionality reduction method you can use
    # for example PCA by projecting on two principal dimensions
    # z = PCA(n=2).fit_transform(z)

    colors = iter(plt.get_cmap('Set1')(np.linspace(0, 1.0, len(classes))))
    for c in classes:
        ax.scatter(*z[y.numpy() == c].T, c=next(colors), marker='o')

    ax.legend(classes)

    ######## Inputs ########
    ax = axarr[1, 0]
    ax.set_title('Inputs')
    ax.axis('off')

    rows = 8
    columns = batch_size // rows

    canvas = np.zeros((IMG_SIZE*rows, columns*IMG_SIZE))
    for i in range(rows):
        for j in range(columns):
            idx = i % columns + rows * j
            canvas[i*IMG_SIZE:(i+1)*IMG_SIZE, j*IMG_SIZE:(j+1)*IMG_SIZE] = x[idx].reshape((IMG_SIZE, IMG_SIZE))
    ax.imshow(canvas, cmap='gray')

    ######## Reconstructions ########
    ax = axarr[1, 1]
    ax.set_title('Reconstructions')
    ax.axis('off')

    canvas = np.zeros((IMG_SIZE*rows, columns*IMG_SIZE))
    for i in range(rows):
        for j in range(columns):
            idx = i % columns + rows * j
            canvas[i*IMG_SIZE:(i+1)*IMG_SIZE, j*IMG_SIZE:(j+1)*IMG_SIZE] = x_hat[idx].reshape((IMG_SIZE, IMG_SIZE))

    ax.imshow(canvas, cmap='gray')

    plt.savefig(tmp_img)
    plt.close(f)
    display(Image(filename=tmp_img))
    clear_output(wait=True)

    os.remove(tmp_img)